# Análise Exploratória de Dados (EDA) - Olist E-Commerce Dataset

Este notebook consolida, limpa e analisa um conjunto de dados de e-commerce da Olist. O objetivo é extrair insights sobre vendas, desempenho de entrega e satisfação do cliente.

**Estrutura do Projeto:**
1.  **Setup do Ambiente**: Importação de bibliotecas e configurações.
2.  **Carregamento e Consolidação**: Funções para carregar e unir os múltiplos arquivos CSV.
3.  **Limpeza e Pré-processamento**: Função para tratar valores ausentes, converter tipos de dados e remover duplicatas.
4.  **Engenharia de Atributos**: Criação de novas colunas (features) para uma análise mais profunda.
5.  **Análise Exploratória e Visualização**: Funções para gerar gráficos e estatísticas descritivas.
6.  **Execução Principal**: Chamada sequencial de todas as etapas da análise.

## 1. Setup do Ambiente

Nesta etapa, importamos todas as bibliotecas necessárias para a análise e configuramos os padrões visuais para os gráficos.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configurações de visualização e avisos
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## 2. Carregamento e Consolidação dos Dados

As funções abaixo são responsáveis por encontrar os arquivos `.csv` em um diretório e uni-los em um único DataFrame, criando uma visão consolidada do negócio.

In [ ]:
def load_data(data_folder: str) -> dict:
    """
    Carrega todos os arquivos CSV do dataset Olist de um diretório especificado.

    Args:
        data_folder (str): O caminho para a pasta que contém os arquivos .csv.

    Returns:
        dict: Um dicionário onde as chaves são os nomes dos datasets e os
              valores são os DataFrames do pandas correspondentes.
    """
    datasets = {
        "customers": "olist_customers_dataset.csv",
        "order_items": "olist_order_items_dataset.csv",
        "payments": "olist_order_payments_dataset.csv",
        "reviews": "olist_order_reviews_dataset.csv",
        "orders": "olist_orders_dataset.csv",
        "products": "olist_products_dataset.csv",
        "sellers": "olist_sellers_dataset.csv",
        "product_category": "product_category_name_translation.csv",
    }

    dataframes = {}
    print("Iniciando carregamento dos dados...")
    for name, filename in datasets.items():
        file_path = os.path.join(data_folder, filename)
        try:
            dataframes[name] = pd.read_csv(file_path)
            print(f"  - Arquivo '{filename}' carregado com sucesso.")
        except FileNotFoundError:
            print(f"  - [ERRO] Arquivo não encontrado: {file_path}")
            return None
    return dataframes

def merge_data(dataframes: dict) -> pd.DataFrame:
    """
    Realiza o merge sequencial dos dataframes da Olist em um único dataframe consolidado.

    Args:
        dataframes (dict): Dicionário de dataframes carregados pela função load_data.

    Returns:
        pd.DataFrame: Um único dataframe contendo todos os dados unidos.
    """
    if not dataframes:
        return None

    print("\nIniciando merge dos dataframes...")
    # O dataframe de 'orders' é a tabela central
    df = pd.merge(dataframes['orders'], dataframes['customers'], on='customer_id', how='left')
    df = pd.merge(df, dataframes['order_items'], on='order_id', how='left')
    df = pd.merge(df, dataframes['payments'], on='order_id', how='left')
    df = pd.merge(df, dataframes['reviews'], on='order_id', how='left')
    df = pd.merge(df, dataframes['products'], on='product_id', how='left')
    df = pd.merge(df, dataframes['sellers'], on='seller_id', how='left')
    df = pd.merge(df, dataframes['product_category'], on='product_category_name', how='left')

    print(f"Merge concluído. DataFrame consolidado com {df.shape[0]} linhas e {df.shape[1]} colunas.")
    return df

## 3. Limpeza e Pré-processamento

Dados brutos raramente estão prontos para análise. Esta função executa as etapas essenciais de limpeza:
- Converte colunas de texto para o formato de data/hora (`datetime`).
- Aplica estratégias para tratar valores ausentes (`NaN`).
- Remove quaisquer linhas duplicadas que possam existir.

In [ ]:
def clean_and_preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Executa a limpeza completa do dataframe: conversão de tipos, tratamento de nulos e duplicatas.

    Args:
        df (pd.DataFrame): O dataframe consolidado.

    Returns:
        pd.DataFrame: O dataframe limpo e pré-processado.
    """
    print("\nIniciando limpeza e pré-processamento...")

    # --- 1. Conversão de Tipos de Dados (Datas) ---
    date_cols = [
        'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
        'order_delivered_customer_date', 'order_estimated_delivery_date',
        'review_creation_date', 'review_answer_timestamp', 'shipping_limit_date'
    ]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    print(f"  - {len(date_cols)} colunas de data convertidas para datetime.")

    # --- 2. Tratamento de Valores Ausentes ---
    numeric_product_cols = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty']
    for col in numeric_product_cols:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

    df['product_category_name'].fillna('unknown', inplace=True)
    df['product_category_name_english'].fillna('unknown', inplace=True)

    df.dropna(subset=['order_approved_at', 'order_delivered_customer_date'], inplace=True)
    print("  - Valores ausentes tratados.")

    # --- 3. Remoção de Duplicatas ---
    initial_rows = df.shape[0]
    df.drop_duplicates(inplace=True)
    rows_removed = initial_rows - df.shape[0]
    if rows_removed > 0:
        print(f"  - {rows_removed} linhas duplicadas foram removidas.")
    else:
        print("  - Nenhuma linha duplicada encontrada.")

    print(f"Limpeza concluída. DataFrame final com {df.shape[0]} linhas.")
    return df

## 4. Engenharia de Atributos

Esta é uma das etapas mais importantes da análise. Criamos novas colunas (atributos) a partir dos dados existentes para permitir insights mais profundos. Aqui, focamos em métricas de tempo de entrega e extraímos componentes das datas.

In [ ]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cria novos atributos úteis a partir dos dados existentes.

    Args:
        df (pd.DataFrame): O dataframe limpo.

    Returns:
        pd.DataFrame: O dataframe com os novos atributos.
    """
    print("\nIniciando engenharia de atributos...")

    # --- Atributos Temporais ---
    # Tempo de entrega (em dias)
    df['delivery_time_days'] = (df['order_delivered_customer_date'] - df['order_approved_at']).dt.days

    # Diferença entre entrega estimada e real (em dias)
    df['delivery_delta_days'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.days

    # Indicador de atraso na entrega
    df['is_late'] = (df['delivery_delta_days'] < 0).astype(int)

    # Extração de componentes da data da compra
    df['purchase_year'] = df['order_purchase_timestamp'].dt.year
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_day_of_week'] = df['order_purchase_timestamp'].dt.day_name()

    print("  - Novos atributos criados: 'delivery_time_days', 'delivery_delta_days', 'is_late', e componentes de data.")
    return df

## 5. Análise Exploratória e Visualização (EDA)

Com os dados limpos e enriquecidos, esta função gera as principais visualizações para responder a perguntas de negócio, como:
- Qual é a tendência de vendas ao longo do tempo?
- Quais são as categorias de produtos mais populares?
- Os clientes estão satisfeitos? 
- O tempo de entrega influencia a satisfação do cliente?

In [ ]:
def perform_exploratory_data_analysis(df: pd.DataFrame):
    """
    Realiza e exibe a análise exploratória dos dados através de estatísticas e gráficos.

    Args:
        df (pd.DataFrame): O dataframe com todos os atributos.
    """
    print("\n--- INICIANDO ANÁLISE EXPLORATÓRIA (EDA) ---")

    # --- Análise 1: Tendência de Vendas ao Longo do Tempo ---
    plt.figure(figsize=(14, 7))
    sales_by_month = df.set_index('order_purchase_timestamp')['payment_value'].resample('M').sum()
    sales_by_month.plot(kind='line', marker='o')
    plt.title('Tendência Mensal de Vendas (Valor Total)', fontsize=16)
    plt.xlabel('Mês da Compra')
    plt.ylabel('Valor Total de Pagamento (R$)')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    # --- Análise 2: Top 10 Categorias de Produtos Mais Vendidas ---
    plt.figure(figsize=(14, 7))
    top_categories = df['product_category_name_english'].value_counts().nlargest(10)
    sns.barplot(x=top_categories.values, y=top_categories.index, palette='viridis')
    plt.title('Top 10 Categorias de Produtos Mais Vendidas', fontsize=16)
    plt.xlabel('Número de Pedidos')
    plt.ylabel('Categoria do Produto (Inglês)')
    plt.show()

    # --- Análise 3: Distribuição das Notas de Avaliação (Review Scores) ---
    plt.figure(figsize=(10, 6))
    sns.countplot(x='review_score', data=df, palette='rocket')
    plt.title('Distribuição das Notas de Avaliação (Review Scores)', fontsize=16)
    plt.xlabel('Nota da Avaliação')
    plt.ylabel('Contagem de Pedidos')
    plt.show()

    # --- Análise 4: Relação entre Tempo de Entrega e Nota de Avaliação ---
    plt.figure(figsize=(12, 7))
    sns.boxplot(x='review_score', y='delivery_time_days', data=df, palette='coolwarm')
    plt.title('Tempo de Entrega (dias) por Nota de Avaliação', fontsize=16)
    plt.xlabel('Nota da Avaliação')
    plt.ylabel('Tempo de Entrega (dias)')
    plt.ylim(0, df['delivery_time_days'].quantile(0.95)) # Remove outliers extremos para melhor visualização
    plt.show()

    # --- Análise 5: Matriz de Correlação de Variáveis Numéricas ---
    cols_to_correlate = [
        'price', 'freight_value', 'payment_value', 'review_score',
        'delivery_time_days', 'delivery_delta_days', 'is_late'
    ]
    corr_matrix = df[cols_to_correlate].corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
    plt.title('Matriz de Correlação entre Variáveis Chave', fontsize=16)
    plt.show()

## 6. Execução Principal

Esta é a célula final que orquestra todo o fluxo. Ela define o caminho para os dados e chama cada uma das funções definidas anteriormente na ordem correta.

In [ ]:
# ATENÇÃO: Substitua pelo caminho correto da sua pasta de dados.
# Se estiver usando o Google Colab, você precisará montar seu Drive primeiro.
data_folder = '/content/drive/MyDrive/ANALISTA DE DADOS/portifolio/data'

# Etapa 1: Carregar os dados
dataframes = load_data(data_folder)

if dataframes:
    # Etapa 2: Unificar os dataframes
    df_merged = merge_data(dataframes)

    # Etapa 3: Limpeza e Pré-processamento
    df_clean = clean_and_preprocess_data(df_merged)

    # Etapa 4: Engenharia de Atributos
    df_final = feature_engineering(df_clean)

    # Exibe as 5 primeiras linhas do dataframe final para verificação
    print("\n--- Amostra do DataFrame Final ---")
    display(df_final.head())
    
    # Etapa 5: Análise Exploratória
    perform_exploratory_data_analysis(df_final)

    print("\nAnálise concluída com sucesso!")